<a href="https://colab.research.google.com/github/diptoromeo/BERT-and-ELMo-with-CNN-BiLSTM-and-CNN_BiLSTM/blob/main/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install allennlp
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 33.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
import random
import keras.models
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import os
import pandas as pd
import regex
import numpy as np
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Lambda, Input, TimeDistributed, LSTM, Bidirectional
from keras.layers import Conv1D, Flatten, Dropout, GlobalMaxPool1D, MaxPooling1D
#from tensorflow.keras.layers import Conv1D, Flatten, Dropout, GlobalMaxPool1D, MaxPooling1D, Dense
import tensorflow_hub as hub
from tensorflow.keras import layers, models, Sequential
import tensorflow.compat.v1 as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.python.client import device_lib
import timeit
tf.compat.v1.experimental.output_all_intermediates(True) #Graph execution for BiLSTM Models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Embedding, Dropout
print(tf.__version__)
print(device_lib.list_local_devices())



2.15.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14005374764160712546
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7722961162730036849
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


#Heading 1
Preprocessing the Dataset

In [3]:
##===============================Nltk abstract_words tokenize======================================
with open('/content/FGFSJournal.txt', 'rt', encoding='UTF8') as file:
    FGCS_abstract = []
    for line in file:
        if '<abstract>' in line:
            abstract = line.split('</abstract>')[0].split('<abstract>')[-1]
            abstract = ''.join(i for i in abstract if not i.isdigit())
            abstract = regex.sub('[^\w\d\s]+', '', abstract)
            ##abstract = nltk.sent_tokenize(abstract)
            abstract = nltk.word_tokenize(abstract)
            stop_words = set(stopwords.words('english'))
            filtered_sentence_abstract = [w.lower() for w in abstract if
                                          w.lower() not in punctuation and w.lower() not in stop_words]
            tagged_list = nltk.pos_tag(filtered_sentence_abstract)
            nouns_list = [t[0] for t in tagged_list if t[-1] == 'NN']
            lm = WordNetLemmatizer()
            singluar_form = [lm.lemmatize(w, pos='v') for w in nouns_list]
            FGCS_abstract.append(singluar_form)

print(len(FGCS_abstract))
print(FGCS_abstract[0])

5659
['architecture', 'inference']


#Heading 2
Spling the preprocessing dataset to Train and Test data.

In [4]:
random.shuffle(FGCS_abstract)
train_data = FGCS_abstract[:4527]
test_data = FGCS_abstract[4527:]
print("train data:", len(train_data))
print("test data:", len(test_data))

train, valid = train_test_split(train_data, test_size=0.25, random_state=42)
print("train:", len(train))
print("valid:", len(valid))

train data: 4527
test data: 1132
train: 3395
valid: 1132


#Data Labeling
Making Data Labeling about Top-10, 20, 30 words

In [5]:
#======================================================train_labels==========================================================================
ten_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud']
twenty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource']
thirty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource',
               'environment', 'algorithm', 'energy', 'management', 'architecture', 'access', 'scheme', 'communication', 'execution', 'order']



##==============================10-words label==================================
train10_labels = []

for doc in train:
    label = []
    for term in ten_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    train10_labels.append(label)

print("10_train labels:", len(train10_labels))

##==================valid_labels==================
valid10_labels = []
for doc in valid:
    label = []
    for term in ten_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    valid10_labels.append(label)

print("10_valid labels:", len(valid10_labels))

##==================Test_labels==================
test10_labels = []
for doc in test_data:
    label = []
    for term in ten_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    test10_labels.append(label)

print("10_test labels:", len(test10_labels))


##==============================20-words label==================================
train20_labels = []
for doc in train:
    label = []
    for term in twenty_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    train20_labels.append(label)

print("20_train labels:", len(train20_labels))


##==================valid_labels==================
valid20_labels = []
for doc in valid:
    label = []
    for term in twenty_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    valid20_labels.append(label)

print("20_valid labels:", len(valid20_labels))


##==================Test_labels==================
test20_labels = []
for doc in test_data:
    label = []
    for term in twenty_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    test20_labels.append(label)

print("20_test labels:", len(test20_labels))


##==============================30-words label==================================
train30_labels = []
for doc in train:
    label = []
    for term in thirty_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    train30_labels.append(label)

print("30_train labels:", len(train30_labels))


##==================valid_labels==================
valid30_labels = []
for doc in valid:
    label = []
    for term in thirty_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    valid30_labels.append(label)

print("30_valid labels:", len(valid30_labels))


##==================Test_labels==================
test30_labels = []
for doc in test_data:
    label = []
    for term in thirty_words:
        if term in doc:
            label.append(1)
        else:
            label.append(0)
    test30_labels.append(label)

print("30_test labels:", len(test30_labels))


10_train labels: 3395
10_valid labels: 1132
10_test labels: 1132
20_train labels: 3395
20_valid labels: 1132
20_test labels: 1132
30_train labels: 3395
30_valid labels: 1132
30_test labels: 1132


#ELMo Embeddings

In [6]:
elmo = hub.load("https://tfhub.dev/google/elmo/3")

#Data Representatiom

In [7]:

# Tokenize and pad sequences
max_words = 100001
max_sequence_length = 156
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
sequences = tokenizer.texts_to_sequences(train_data)
X = pad_sequences(sequences, maxlen=max_sequence_length)

tokenizer.fit_on_texts(test_data)
test = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test, maxlen=max_sequence_length)

y10_train = np.array(train10_labels)
y20_train = np.array(train20_labels)
y30_train = np.array(train30_labels)
y10_valid = np.array(valid10_labels)
y20_valid = np.array(valid20_labels)
y30_valid = np.array(valid30_labels)
y10_test = np.array(test10_labels)
y20_test = np.array(test20_labels)
y30_test = np.array(test30_labels)

y10_test = np.argmax(y10_test, axis=1)
y20_test = np.argmax(y20_test, axis=1)
y30_test = np.argmax(y30_test, axis=1)


# Split the dataset into training and testing sets
X_train, X_valid = train_test_split(X,  test_size=0.25, random_state=42)

vocab_size = min(max_words, len(tokenizer.word_index) + 1)
embedding_dim = 1024  # ELMo embeddings have a dimension of 1024

# Add an Embedding layer to the model
vocab_size = min(max_words, len(tokenizer.word_index) + 1)
embedding_dim = 1024  # This should match the ELMo embedding dimension

#Models Hyperparameters

In [8]:
epochs = 20
batch_sizes = 16
bilstm_embedding_dim = 256
filter_sizes = 128
kernel_sizes = 5
hidden_layers = 64
lstm_memory = 64
bilstm_memory = 50
dropout = 0.5
ten_num_classes = 10
twenty_num_classes = 20
thirty_num_classes = 30

#ELMo + CNN Model

#ELMo Embedding
Making ELMo Embedding to get ELMo from TensoFlow

In [11]:
tf.disable_eager_execution()

In [12]:
def ten_word_cnn_model():
  cnn_model = Sequential()
  cnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
  cnn_model.add(Conv1D(filters=filter_sizes, kernel_size=kernel_sizes, activation='relu'))
  cnn_model.add(GlobalMaxPooling1D())
  cnn_model.add(Dense(hidden_layers, activation='relu'))
  cnn_model.add(Dropout(dropout))
  cnn_model.add(Dense(ten_num_classes, activation='softmax'))
  cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  cnn_model.summary()
  return cnn_model


def twenty_word_cnn_model():
  cnn_model = Sequential()
  cnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
  cnn_model.add(Conv1D(filters=filter_sizes, kernel_size=kernel_sizes, activation='relu'))
  cnn_model.add(GlobalMaxPooling1D())
  cnn_model.add(Dense(hidden_layers, activation='relu'))
  cnn_model.add(Dropout(dropout))
  cnn_model.add(Dense(twenty_num_classes, activation='softmax'))
  cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  cnn_model.summary()
  return cnn_model


def thirty_word_cnn_model():
  cnn_model = Sequential()
  cnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
  cnn_model.add(Conv1D(filters=filter_sizes, kernel_size=kernel_sizes, activation='relu'))
  cnn_model.add(GlobalMaxPooling1D())
  cnn_model.add(Dense(hidden_layers, activation='relu'))
  cnn_model.add(Dropout(dropout))
  cnn_model.add(Dense(thirty_num_classes, activation='softmax'))
  cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  cnn_model.summary()
  return cnn_model


cnn_model1 = ten_word_cnn_model()
cnn_model2 = twenty_word_cnn_model()
cnn_model3 = thirty_word_cnn_model()

cnn_model1.save("ten_word_cnn_model")
cnn_model1.save("twenty_word_cnn_model")
cnn_model1.save("thirty_word_cnn_model")


##=================10words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history10 = cnn_model1.fit(X_train, y10_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo10_CNN_Accuracy = cnn_model1.evaluate(X_train, y10_train, verbose=1)
    print(('ELMo10_CNN_Accuracy: %f' % (ELMo10_CNN_Accuracy[1] * 100)))


##===============================ELMo10_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = cnn_model1.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo10_CNN_Test_Accuracy = accuracy_score(y10_test, np.round(abs(predictions)))
    print('ELMo10_CNN_Test_Accuracy: %f' % ELMo10_CNN_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo10_CNN_Precision = precision_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_CNN_Precision: %f' % ELMo10_CNN_Precision)

    # Calculating the recall score of classifier
    ELMo10_CNN_Recall = recall_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_CNN_Recall: %f' % ELMo10_CNN_Recall)

    # Calculating the F1 score of classifier
    ELMo10_CNN_F1_Score = f1_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_CNN_F1_Score: %f' % ELMo10_CNN_F1_Score)


cnn_data10 = {
    'Name': ["ELMo10_CNN_Accuracy", "ELMo10CNN_Test_Accuracy", "ELMo10_CNN_Precision", "ELMo10_CNN_Recall", "ELMo10_CNN_F1_Score"],
    'Scores': [ELMo10_CNN_Accuracy, ELMo10_CNN_Test_Accuracy, ELMo10_CNN_Precision, ELMo10_CNN_Recall, ELMo10_CNN_F1_Score] #
}
df = pd.DataFrame(cnn_data10)
df.to_excel('ELMo10_CNN_Scores_file.xlsx', index=False)


##=================20words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history20 = cnn_model2.fit(X_train, y20_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo20_CNN_Accuracy = cnn_model2.evaluate(X_train, y20_train, verbose=1)
    print(('ELMo20_CNN_Accuracy: %f' % (ELMo20_CNN_Accuracy[1] * 100)))

##===============================ELMo20_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = cnn_model2.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo20_CNN_Test_Accuracy = accuracy_score(y20_test, np.round(abs(predictions)))
    print('ELMo20_CNN_Test_Accuracy y: %f' % ELMo20_CNN_Test_Accuracy )

    # Calculating the precision score of classifier
    ELMo20_CNN_Precision = precision_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_CNN_Precision: %f' % ELMo20_CNN_Precision)

    # Calculating the recall score of classifier
    ELMo20_CNN_Recall = recall_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_CNN_Recall: %f' % ELMo20_CNN_Recall)

    # Calculating the F1 score of classifier
    ELMo20_CNN_F1_Score = f1_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_CNN_F1_Score: %f' % ELMo20_CNN_F1_Score)


cnn_data20 = {
    'Name': ["ELMo20_CNN_Accuracy", "ELMo20_CNN_Test_Accuracy ", "ELMo20CNN_Precision", "ELMo20_CNN_Recall", "ELMo20_CNN_F1_Score"],
    'Scores': [ELMo20_CNN_Accuracy, ELMo20_CNN_Test_Accuracy , ELMo20_CNN_Precision, ELMo20_CNN_Recall, ELMo20_CNN_F1_Score]
}
df = pd.DataFrame(cnn_data20)
df.to_excel('ELMo20_CNN_Scores_file.xlsx', index=False)


##=================30words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history30 = cnn_model3.fit(X_train, y30_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo30_CNN_Accuracy = cnn_model1.evaluate(X_train, y10_train, verbose=1)
    print(('ELMo30_CNN_Accuracy: %f' % (ELMo30_CNN_Accuracy[1] * 100)))

##===============================ELMo30_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = cnn_model3.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo30_CNN_Test_Accuracy = accuracy_score(y30_test, np.round(abs(predictions)))
    print('ELMo30_CNN_Test_Accuracy: %f' % ELMo30_CNN_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo30_CNN_Precision = precision_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_CNN_Precision: %f' % ELMo30_CNN_Precision)

    # Calculating the recall score of classifier
    ELMo30_CNN_Recall = recall_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_CNN_Recall: %f' % ELMo30_CNN_Recall)

    # Calculating the F1 score of classifier
    ELMo30_CNN_F1_Score = f1_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_CNN_F1_Score: %f' % ELMo30_CNN_F1_Score)


cnn_data30 = {
    'Name': ["ELMo30_CNN_F1_Score", "ELMo30_CNN_Test_Accuracy", "ELMo30_CNN_Precision", "ELMo30_CNN_Recall", "ELMo30_CNN_F1_Score"],
    'Scores': [ELMo30_CNN_F1_Score, ELMo30_CNN_Test_Accuracy, ELMo30_CNN_Precision, ELMo30_CNN_Recall, ELMo30_CNN_F1_Score]
}
df = pd.DataFrame(cnn_data30)
df.to_excel('ELMo30_CNN_Scores_file.xlsx', index=False)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 156, 1024)         13281280  
                                                                 
 conv1d_3 (Conv1D)           (None, 152, 128)          655488    
                                                                 
 global_max_pooling1d_3 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 10)                650       
                                                      

Train on 8 samples
Epoch 1/20
8/8 [==============================] - 24s 1s/step - batch: 3.5000 - size: 1.0000 - loss: 0.6186 - accuracy: 0.7096
Epoch 2/20
8/8 [==============================] - 9s 1s/step - batch: 3.5000 - size: 1.0000 - loss: 0.5209 - accuracy: 0.7213
Epoch 3/20
8/8 [==============================] - 9s 1s/step - batch: 3.5000 - size: 1.0000 - loss: 0.4205 - accuracy: 0.7381
Epoch 4/20
8/8 [==============================] - 9s 1s/step - batch: 3.5000 - size: 1.0000 - loss: 0.3236 - accuracy: 0.7656
Epoch 5/20
8/8 [==============================] - 9s 1s/step - batch: 3.5000 - size: 1.0000 - loss: 0.2538 - accuracy: 0.7799
Epoch 6/20
8/8 [==============================] - 10s 1s/step - batch: 3.5000 - size: 1.0000 - loss: 0.2048 - accuracy: 0.7862
Epoch 7/20
8/8 [==============================] - 10s 1s/step - batch: 3.5000 - size: 1.0000 - loss: 0.1723 - accuracy: 0.7894
Epoch 8/20
8/8 [==============================] - 10s 1s/step - batch: 3.5000 - size: 1.0000 - l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


ELMo10_CNN_Accuracy: 78.114891


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


ELMo20_CNN_Test_Accuracy y: 0.654594
ELMo20_CNN_Precision: 0.654594
ELMo20_CNN_Recall: 0.654594
ELMo20_CNN_F1_Score: 0.654594


Train on 8 samples
Epoch 1/20
8/8 [==============================] - 6s 585ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.6570 - accuracy: 0.8031
Epoch 2/20
8/8 [==============================] - 5s 587ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5920 - accuracy: 0.8034
Epoch 3/20
8/8 [==============================] - 5s 591ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5470 - accuracy: 0.8040
Epoch 4/20
8/8 [==============================] - 5s 593ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5030 - accuracy: 0.8060
Epoch 5/20
8/8 [==============================] - 5s 596ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4581 - accuracy: 0.8078
Epoch 6/20
8/8 [==============================] - 5s 596ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4152 - accuracy: 0.8089
Epoch 7/20
8/8 [==============================] - 5s 595ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.3780 - accuracy: 0.8100
Epoch 8/20
8/8 [==============================] - 5s 593ms/step - batch: 3.500

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


ELMo30_CNN_Test_Accuracy: 0.649293
ELMo30_CNN_Precision: 0.649293
ELMo30_CNN_Recall: 0.649293
ELMo30_CNN_F1_Score: 0.649293


#ELMo + BiLSTM Models

In [13]:
%env TF_ALLOCATOR_MAX_ALLOCATION_SIZE=10G

env: TF_ALLOCATOR_MAX_ALLOCATION_SIZE=10G


In [14]:
from optparse import Option
# Tuning and Define the BiLSTM models
batch_sizes = 16

def ten_word_bilstm_model():
  bilstm_model = Sequential()
  bilstm_model.add(Embedding(input_dim=vocab_size, output_dim=bilstm_embedding_dim, input_length=max_sequence_length))
  bilstm_model.add(Bidirectional(LSTM(bilstm_memory, return_sequences = False)))
  bilstm_model.add(Dropout(dropout))
  bilstm_model.add(Dense(ten_num_classes, activation='softmax'))
  bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  bilstm_model.summary()
  return bilstm_model

def twenty_word_bilstm_model():
  bilstm_model = Sequential()
  bilstm_model.add(Embedding(input_dim=vocab_size, output_dim=bilstm_embedding_dim, input_length=max_sequence_length))
  bilstm_model.add(Bidirectional(LSTM(bilstm_memory, return_sequences = False)))
  bilstm_model.add(Dropout(dropout))
  bilstm_model.add(Dense(twenty_num_classes, activation='softmax'))
  bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  bilstm_model.summary()
  return bilstm_model

def thirty_word_bilstm_model():
  bilstm_model = Sequential()
  bilstm_model.add(Embedding(input_dim=vocab_size, output_dim=bilstm_embedding_dim, input_length=max_sequence_length))
  bilstm_model.add(Bidirectional(LSTM(bilstm_memory, return_sequences = False)))
  bilstm_model.add(Dropout(dropout))
  bilstm_model.add(Dense(thirty_num_classes, activation='softmax'))
  # Compile the model
  bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  bilstm_model.summary()
  return bilstm_model


bilstm_model1 = ten_word_bilstm_model()
bilstm_model2 = twenty_word_bilstm_model()
bilstm_model3 = thirty_word_bilstm_model()

bilstm_model1.save("ten_word_bilstm_model")
bilstm_model1.save("twenty_word_bilstm_model")
bilstm_model1.save("thirty_word_bilstm_model")


##=================10words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history10 = bilstm_model1.fit(X_train, y10_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo10_BiLSTM_Accuracy = bilstm_model1.evaluate(X_train, y10_train, verbose=1)
    print(('ELMo10_BiLSTM_Accuracy: %f' % (ELMo10_BiLSTM_Accuracy[1] * 100)))


##===============================ELMo10_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = bilstm_model1.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo10_BiLSTM_Test_Accuracy = accuracy_score(y10_test, np.round(abs(predictions)))
    print('ELMo10_BiLSTM_Test_Accuracy: %f' % ELMo10_BiLSTM_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo10_BiLSTM_precision = precision_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_BiLSTM_precision: %f' % ELMo10_BiLSTM_precision)

    # Calculating the recall score of classifier
    ELMo10_BiLSTM_Recall = recall_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_BiLSTM_Recall: %f' % ELMo10_BiLSTM_Recall)

    # Calculating the F1 score of classifier
    ELMo10_BiLSTM_F1_Score = f1_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_BiLSTM_F1_Score: %f' % ELMo10_BiLSTM_F1_Score)


bilstm_data10 = {
    'Name': ["ELMo10_BiLSTM_Accuracy", "ELMo10_BiLSTM_Test_Accuracy", "ELMo10_BiLSTM_precision", "ELMo10_BiLSTM_Recall", "ELMo10_BiLSTM_F1_Score"],
    'Scores': [ELMo10_BiLSTM_Accuracy, ELMo10_BiLSTM_Test_Accuracy, ELMo10_BiLSTM_precision, ELMo10_BiLSTM_Recall, ELMo10_BiLSTM_F1_Score]
}
df = pd.DataFrame(bilstm_data10)
df.to_excel('ELMo10_BiLSTM_Scores_file.xlsx', index=False)


##=================20words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history20 = bilstm_model2.fit(X_train, y20_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo20_BiLSTM_Accuracy = bilstm_model2.evaluate(X_train, y20_train, verbose=1)
    print(('ELMo20_BiLSTM_Accuracy: %f' % (ELMo20_BiLSTM_Accuracy[1] * 100)))


##===============================ELMo20_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = bilstm_model2.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo20_BiLSTM_Test_Accuracy = accuracy_score(y20_test, np.round(abs(predictions)))
    print('ELMo20_BiLSTM_Test_Accuracy: %f' % ELMo20_BiLSTM_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo20_BiLSTM_precision = precision_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_BiLSTM_precision: %f' % ELMo20_BiLSTM_precision)

    # Calculating the recall score of classifier
    ELMo20_BiLSTM_Recall = recall_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_BiLSTM_Recall: %f' % ELMo20_BiLSTM_Recall)

    # Calculating the F1 score of classifier
    ELMo20_BiLSTM_F1_Score = f1_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_BiLSTM_F1_Score: %f' % ELMo20_BiLSTM_F1_Score)


bilstm_data20 = {
    'Name': ["ELMo20_BiLSTM_Accuracy", "ELMo20_BiLSTM_Test_Accuracy", "ELMo20_BiLSTM_precision", "ELMo20_BiLSTM_Recall", "ELMo20_BiLSTM_F1_Score"],
    'Scores': [ELMo20_BiLSTM_Accuracy, ELMo20_BiLSTM_Test_Accuracy, ELMo20_BiLSTM_precision, ELMo20_BiLSTM_Recall, ELMo20_BiLSTM_F1_Score]
}
df = pd.DataFrame(bilstm_data20)
df.to_excel('ELMo20_BiLSTM_Scores_file.xlsx', index=False)


##=================30words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history30 = bilstm_model3.fit(X_train, y30_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo30_BiLSTM_Accuracy = bilstm_model3.evaluate(X_train, y30_train, verbose=1)
    print(('ELMo30_BiLSTM_Accuracy: %f' % (ELMo30_BiLSTM_Accuracy[1] * 100)))


##===============================ELMo30_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = bilstm_model3.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo30_BiLSTM_Test_Accuracy = accuracy_score(y30_test, np.round(abs(predictions)))
    print('ELMo30_BiLSTM_Test_Accuracy: %f' % ELMo30_BiLSTM_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo30_BiLSTM_precision = precision_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_BiLSTM_precision: %f' % ELMo30_BiLSTM_precision)

    # Calculating the recall score of classifier
    ELMo30_BiLSTM_Recall = recall_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_BiLSTM_Recall: %f' % ELMo30_BiLSTM_Recall)

    # Calculating the F1 score of classifier
    ELMo30_BiLSTM_F1_Score = f1_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_BiLSTM_F1_Score: %f' % ELMo30_BiLSTM_F1_Score)


bilstm_data30 = {
    'Name': ["ELMo30_BiLSTM_Accuracy", "ELMo30_BiLSTM_Test_Accuracy", "ELMo30_BiLSTM_precision", "ELMo30_BiLSTM_Recall", "ELMo30_BiLSTM_F1_Score"],
    'Scores': [ELMo30_BiLSTM_Accuracy, ELMo30_BiLSTM_Test_Accuracy, ELMo30_BiLSTM_precision, ELMo30_BiLSTM_Recall, ELMo30_BiLSTM_F1_Score]
}
df = pd.DataFrame(bilstm_data30)
df.to_excel('ELMo30_BiLSTM_Scores_file.xlsx', index=False)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 156, 256)          3320320   
                                                                 
 bidirectional (Bidirection  (None, 100)               122800    
 al)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_12 (Dense)            (None, 10)                1010      
                                                                 
Total params: 3444130 (13.14 MB)
Trainable params: 3444130 (13.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 156, 256)          3320320   
                                                                 
 bidirectional_1 (Bidirecti  (None, 100)               122800    
 onal)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_13 (Dense)            (None, 20)                2020      
                                                                 
Total params: 3445140 (13.14 MB)
Trainable params: 3445140 (13.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 156, 256)          3320320   
                                                                 
 bidirectional_2 (Bidirecti  (None, 100)               122800    
 onal)                                                           
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_14 (Dense)            (None, 30)                3030      
                                                                 
Total params: 3446150 (13.15 MB)
Trainable params: 3446150 (13.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Train on 8 samples
Epoch 1/20
8/8 [==============================] - 6s 487ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.6729 - accuracy: 0.7081
Epoch 2/20
8/8 [==============================] - 4s 488ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.6051 - accuracy: 0.7088
Epoch 3/20
8/8 [==============================] - 4s 488ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5732 - accuracy: 0.7132
Epoch 4/20
8/8 [==============================] - 4s 487ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5548 - accuracy: 0.7096
Epoch 5/20
8/8 [==============================] - 4s 489ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5240 - accuracy: 0.7180
Epoch 6/20
8/8 [==============================] - 4s 490ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4956 - accuracy: 0.7217
Epoch 7/20
8/8 [==============================] - 4s 489ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4706 - accuracy: 0.7249
Epoch 8/20
8/8 [==============================] - 4s 492ms/step - batch: 3.500

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


...

Train on 8 samples
Epoch 1/20
8/8 [==============================] - 4s 342ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.6709 - accuracy: 0.8031
Epoch 2/20
8/8 [==============================] - 3s 343ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5782 - accuracy: 0.8031
Epoch 3/20
8/8 [==============================] - 3s 343ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5110 - accuracy: 0.8037
Epoch 4/20
8/8 [==============================] - 3s 344ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4923 - accuracy: 0.8032
Epoch 5/20
8/8 [==============================] - 3s 347ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4851 - accuracy: 0.8031
Epoch 6/20
8/8 [==============================] - 3s 346ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4801 - accuracy: 0.8031
Epoch 7/20
8/8 [==============================] - 3s 348ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4726 - accuracy: 0.8031
Epoch 8/20
8/8 [==============================] - 3s 349ms/step - batch: 3.500

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


ELMo30_BiLSTM_Accuracy: 80.443805


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


ELMo30_BiLSTM_Test_Accuracy: 0.649293
ELMo30_BiLSTM_precision: 0.649293
ELMo30_BiLSTM_Recall: 0.649293
ELMo30_BiLSTM_F1_Score: 0.649293


#ELMo+CNN_BiLSTM Models

In [ ]:
tf.disable_eager_execution()

In [15]:
def ten_word_cnn_bilstm_model():
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=bilstm_embedding_dim, input_length=max_sequence_length))
  model.add(Conv1D(128, kernel_size = 5, input_shape = (max_sequence_length, vocab_size), activation = 'relu'))
  model.add(MaxPooling1D())
  model.add(Dropout(dropout))
  model.add(Bidirectional(LSTM(64, return_sequences=False)))
  model.add(Dense(ten_num_classes, activation = 'softmax'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  model.summary()
  return model

def twenty_word_cnn_bilstm_model():
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=bilstm_embedding_dim, input_length=max_sequence_length))
  model.add(Conv1D(filter_sizes, kernel_size = kernel_sizes, input_shape = (max_sequence_length, vocab_size), activation = 'relu'))
  model.add(MaxPooling1D())
  model.add(Dropout(dropout))
  model.add(Bidirectional(LSTM(lstm_memory, return_sequences=False)))
  model.add(Dense(twenty_num_classes, activation = 'softmax'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  model.summary()
  return model

def thirty_word_cnn_bilstm_model():
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=bilstm_embedding_dim, input_length=max_sequence_length))
  model.add(Conv1D(filter_sizes, kernel_size = kernel_sizes, input_shape = (max_sequence_length, vocab_size), activation = 'relu'))
  model.add(MaxPooling1D())
  model.add(Dropout(dropout))
  model.add(Bidirectional(LSTM(lstm_memory, return_sequences=False)))
  model.add(Dense(thirty_num_classes, activation = 'softmax'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  model.summary()
  return model


cnn_bilstm_model1 = ten_word_cnn_bilstm_model()
cnn_bilstm_model2 = twenty_word_cnn_bilstm_model()
cnn_bilstm_model3 = thirty_word_cnn_bilstm_model()

cnn_bilstm_model1.save('ten_word_cnn_bilstm_model')
cnn_bilstm_model2.save('twenty_word_cnn_bilstm_model')
cnn_bilstm_model3.save('thirty_word_cnn_bilstm_model')


##=================10words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history10 = cnn_bilstm_model1.fit(X_train, y10_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo10_CNN_BiLSTM_Accuracy = cnn_bilstm_model1.evaluate(X_train, y10_train, verbose=1)
    print(('ELMo10_CNN_BiLSTM_Accuracy: %f' % (ELMo10_CNN_BiLSTM_Accuracy[1] * 100)))

##===============================ELMo10_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = cnn_bilstm_model1.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo10_CNN_BiLSTM_Test_Accuracy = accuracy_score(y10_test, np.round(abs(predictions)))
    print('ELMo10_CNN_BiLSTM_Test_Accuracy: %f' % ELMo10_CNN_BiLSTM_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo10_CNN_BiLSTM_precision = precision_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_CNN_BiLSTM_precision: %f' % ELMo10_CNN_BiLSTM_precision)

    # Calculating the recall score of classifier
    ELMo10_CNN_BiLSTM_Recall = recall_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_CNN_BiLSTM_Recall: %f' % ELMo10_CNN_BiLSTM_Recall)

    # Calculating the F1 score of classifier
    ELMo10_CNN_BiLSTM_F1_Score = f1_score(y10_test, np.round(abs(predictions)), average='micro')
    print('ELMo10_CNN_BiLSTM_F1_Score: %f' % ELMo10_CNN_BiLSTM_F1_Score)


cnn_bilstm_data10 = {
    'Name': ["ELMo10_CNN_BiLSTM_Accuracy", "ELMo10_CNN_BiLSTM_Test_Accuracy", "ELMo10_CNN_BiLSTM_precision", "ELMo10_CNN_BiLSTM_Recall", "ELMo10_CNN_BiLSTM_F1_Score"],
    'Scores': [ELMo10_CNN_BiLSTM_Accuracy, ELMo10_CNN_BiLSTM_Test_Accuracy, ELMo10_CNN_BiLSTM_precision, ELMo10_CNN_BiLSTM_Recall, ELMo10_CNN_BiLSTM_F1_Score]
}
df = pd.DataFrame(cnn_bilstm_data10)
df.to_excel('ELMo10_CNN_BiLSTM_Scores_file.xlsx', index=False)



##=================20words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history20 = cnn_bilstm_model2.fit(X_train, y20_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo20_CNN_BiLSTM_Accuracy = cnn_bilstm_model2.evaluate(X_train, y20_train, verbose=1)
    print(('ELMo20_CNN_BiLSTM_Accuracy: %f' % (ELMo20_CNN_BiLSTM_Accuracy[1] * 100)))

##===============================ELMo20_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = cnn_bilstm_model2.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo20_CNN_BiLSTM_Test_Accuracy = accuracy_score(y20_test, np.round(abs(predictions)))
    print('ELMo20_CNN_BiLSTM_Test_Accuracy: %f' % ELMo20_CNN_BiLSTM_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo20_CNN_BiLSTM_precision = precision_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_CNN_BiLSTM__precision: %f' % ELMo20_CNN_BiLSTM_precision)

    # Calculating the recall score of classifier
    ELMo20_CNN_BiLSTM_Recall = recall_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_CNN_BiLSTM_Recall: %f' % ELMo20_CNN_BiLSTM_Recall)

    # Calculating the F1 score of classifier
    ELMo20_CNN_BiLSTM_F1_Score = f1_score(y20_test, np.round(abs(predictions)), average='micro')
    print('ELMo20_CNN_BiLSTM_F1_Score: %f' % ELMo20_CNN_BiLSTM_F1_Score)


cnn_bilstm_data20 = {
    'Name': ["ELMo20_CNN_BiLSTM_Accuracy", "ELMo20_CNN_BiLSTM_Test_Accuracy", "ELMo20_CNN_BiLSTM_precision", "ELMo20_CNN_BiLSTM_Recall", "ELMo20_CNN_BiLSTM_F1_Score"],
    'Scores': [ELMo20_CNN_BiLSTM_Accuracy, ELMo20_CNN_BiLSTM_Test_Accuracy, ELMo20_CNN_BiLSTM_precision, ELMo20_CNN_BiLSTM_Recall, ELMo20_CNN_BiLSTM_F1_Score]
}
df = pd.DataFrame(cnn_bilstm_data20)
df.to_excel('ELMo20_CNN_BiLSTM_Scores_file.xlsx', index=False)



##=================30words model training=====================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    history30 = cnn_bilstm_model3.fit(X_train, y30_train, epochs=epochs, batch_size=batch_sizes, steps_per_epoch=8)

    ELMo30_CNN_BiLSTM_Accuracy = cnn_bilstm_model3.evaluate(X_train, y30_train, verbose=1)
    print(('ELMo30_CNN_BiLSTM_Accuracy: %f' % (ELMo30_CNN_BiLSTM_Accuracy[1] * 100)))

##===============================ELMo30_CNN_Model Evaluation========================================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # Here X_test, y_test are the test data points
    predictions = bilstm_model3.predict(X_test)
    predictions = predictions[:, 0]

    # CBOW Tst Accuracy
    ELMo30_CNN_BiLSTM_Test_Accuracy = accuracy_score(y30_test, np.round(abs(predictions)))
    print('ELMo30_CNN_BiLSTM_Test_Accuracy: %f' % ELMo30_CNN_BiLSTM_Test_Accuracy)

    # Calculating the precision score of classifier
    ELMo30_CNN_BiLSTM_precision = precision_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_CNN_BiLSTM_precision: %f' % ELMo30_CNN_BiLSTM_precision)

    # Calculating the recall score of classifier
    ELMo30_CNN_BiLSTM_Recall = recall_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_CNN_BiLSTM_Recall: %f' % ELMo30_CNN_BiLSTM_Recall)

    # Calculating the F1 score of classifier
    ELMo30_CNN_BiLSTM_F1_Score = f1_score(y30_test, np.round(abs(predictions)), average='micro')
    print('ELMo30_CNN_BiLSTM_F1_Score: %f' % ELMo30_CNN_BiLSTM_F1_Score)


cnn_bilstm_data30 = {
    'Name': ["ELMo30_CNN_BiLSTM_Accuracy", "ELMo30_CNN_BiLSTM_Test_Accuracy", "ELMo30_CNN_BiLSTM_precision", "ELMo30_CNN_BiLSTM_Recall", "ELMo30_CNN_BiLSTM_F1_Score"],
    'Scores': [ELMo30_CNN_BiLSTM_Accuracy, ELMo30_CNN_BiLSTM_Test_Accuracy, ELMo30_CNN_BiLSTM_precision, ELMo30_CNN_BiLSTM_Recall, ELMo30_CNN_BiLSTM_F1_Score]
}
df = pd.DataFrame(cnn_bilstm_data30)
df.to_excel('ELMo30_CNN_BiLSTM_Scores_file.xlsx', index=False)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 156, 256)          3320320   
                                                                 
 conv1d_6 (Conv1D)           (None, 152, 128)          163968    
                                                                 
 max_pooling1d (MaxPooling1  (None, 76, 128)           0         
 D)                                                              
                                                                 
 dropout_9 (Dropout)         (None, 76, 128)           0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_15 (Dense)            (None, 10)               

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 156, 256)          3320320   
                                                                 
 conv1d_7 (Conv1D)           (None, 152, 128)          163968    
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 76, 128)           0         
 g1D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 76, 128)           0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_16 (Dense)            (None, 20)              

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 156, 256)          3320320   
                                                                 
 conv1d_8 (Conv1D)           (None, 152, 128)          163968    
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 76, 128)           0         
 g1D)                                                            
                                                                 
 dropout_11 (Dropout)        (None, 76, 128)           0         
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_17 (Dense)            (None, 30)              

Train on 8 samples
Epoch 1/20
8/8 [==============================] - 9s 493ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.6360 - accuracy: 0.7081
Epoch 2/20
8/8 [==============================] - 4s 494ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5645 - accuracy: 0.7083
Epoch 3/20
8/8 [==============================] - 4s 494ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5497 - accuracy: 0.7081
Epoch 4/20
8/8 [==============================] - 4s 495ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5243 - accuracy: 0.7081
Epoch 5/20
8/8 [==============================] - 4s 495ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4766 - accuracy: 0.7087
Epoch 6/20
8/8 [==============================] - 4s 498ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4252 - accuracy: 0.7181
Epoch 7/20
8/8 [==============================] - 4s 496ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.3729 - accuracy: 0.7341
Epoch 8/20
8/8 [==============================] - 4s 497ms/step - batch: 3.500

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


ELMo20_CNN_BiLSTM_Test_Accuracy: 0.654594
ELMo20_CNN_BiLSTM__precision: 0.654594
ELMo20_CNN_BiLSTM_Recall: 0.654594
ELMo20_CNN_BiLSTM_F1_Score: 0.654594


Train on 8 samples
Epoch 1/20
8/8 [==============================] - 6s 493ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.6435 - accuracy: 0.8031
Epoch 2/20
8/8 [==============================] - 4s 493ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.5113 - accuracy: 0.8031
Epoch 3/20
8/8 [==============================] - 4s 496ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4707 - accuracy: 0.8031
Epoch 4/20
8/8 [==============================] - 4s 497ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4673 - accuracy: 0.8031
Epoch 5/20
8/8 [==============================] - 4s 498ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4622 - accuracy: 0.8031
Epoch 6/20
8/8 [==============================] - 4s 499ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4554 - accuracy: 0.8031
Epoch 7/20
8/8 [==============================] - 4s 499ms/step - batch: 3.5000 - size: 1.0000 - loss: 0.4455 - accuracy: 0.8031
Epoch 8/20
8/8 [==============================] - 4s 499ms/step - batch: 3.500

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


ELMo30_CNN_BiLSTM_Accuracy: 81.561118
ELMo30_CNN_BiLSTM_Test_Accuracy: 0.649293
ELMo30_CNN_BiLSTM_precision: 0.649293
ELMo30_CNN_BiLSTM_Recall: 0.649293
ELMo30_CNN_BiLSTM_F1_Score: 0.649293


In [20]:
#======================================================train_labels==========================================================================
ten_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud']
twenty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource']
thirty_words = ['paper', 'system', 'performance', 'network', 'model', 'service', 'time', 'information', 'approach', 'cloud',
                'problem', 'process', 'security', 'analysis', 'application', 'method', 'research', 'framework', 'number', 'resource',
               'environment', 'algorithm', 'energy', 'management', 'architecture', 'access', 'scheme', 'communication', 'execution', 'order']




##==============================10-words label==================================
ten_words_labels = []

for i in range(0, 1132):
    count = 0
    for j in range(0, len(ten_words)):
        if ten_words[j] in test_data[i]:
            count += 1
    if count >=1:
        ten_words_labels.append(1)
    else:
        ten_words_labels.append(0)

print("ten_words_labels:", len(ten_words_labels))

##==============================20-words label==================================
twenty_words_labels = []

for i in range(0, 1132):
    count = 0
    for j in range(0, len(twenty_words)):
        if twenty_words[j] in test_data[i]:
            count += 1
    if count >=1:
        twenty_words_labels.append(1)
    else:
        twenty_words_labels.append(0)

print("twenty_words_labels:", len(twenty_words_labels))

##==============================30-words label==================================
thrity_words_labels = []

for i in range(0, 1132):
    count = 0
    for j in range(0, len(thirty_words)):
        if thirty_words[j] in test_data[i]:
            count += 1
    if count >=1:
        thrity_words_labels.append(1)
    else:
        thrity_words_labels.append(0)

print("thrity_words_labels:", len(thrity_words_labels))


ten_words_labels: 1132
twenty_words_labels: 1132
thrity_words_labels: 1132


In [ ]:
tf.compat.v1.experimental.output_all_intermediates(True) #Graph execution for BiLSTM Models

In [24]:
##============================BILSTM_Model_ROC_AuC_Scores======================================================
positive = 1
negative = 0
none = None
micro = 'micro'
macro = 'macro'
weight = "weighted"


predictions =cnn_model1.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo10_cnn_roc_auc_Score = roc_auc_score(ten_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo10_cnn_roc_auc_Score: %f' % ELMo10_cnn_roc_auc_Score)


# Here X_test, y_test are the test data points
predictions =cnn_model2.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo20_cnn_roc_auc_Score = roc_auc_score(twenty_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo20_cnn_roc_auc_Score: %f' % ELMo20_cnn_roc_auc_Score)

# Here X_test, y_test are the test data points
predictions =cnn_model3.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo30_cnn_roc_auc_Score = roc_auc_score(thrity_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo30_cnn_roc_auc_Score: %f' % ELMo30_cnn_roc_auc_Score)


##============================BILSTM_Model_ROC_AuC_Scores======================================================

# Here X_test, y_test are the test data points
predictions =bilstm_model1.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo10_bilstm_roc_auc_Score = roc_auc_score(ten_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo10_bilstm_roc_auc_Score: %f' % ELMo10_bilstm_roc_auc_Score)

# Here X_test, y_test are the test data points
predictions =bilstm_model2.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo20_bilstm_roc_auc_Score = roc_auc_score(twenty_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo20_bilstm_roc_auc_Score: %f' % ELMo20_bilstm_roc_auc_Score)

# Here X_test, y_test are the test data points
predictions =bilstm_model3.predict(X_test)
predictions = predictions[:, positive]
#Calculating the F1 score of classifier
ELMo30_bilstm_roc_auc_Score = roc_auc_score(thrity_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo30_bilstm_roc_auc_Score: %f' % ELMo30_bilstm_roc_auc_Score)

##============================CNN_BILSTM_Model_ROC_AuC_Scores======================================================
# Here X_test, y_test are the test data points
predictions =cnn_bilstm_model1.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo10_cnn_bilstm_roc_auc_Score = roc_auc_score(ten_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo10_cnn_bilstm_roc_auc_Score: %f' % ELMo10_cnn_bilstm_roc_auc_Score)

# Here X_test, y_test are the test data points
predictions =cnn_bilstm_model2.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo20_cnn_bilstm_roc_auc_Score = roc_auc_score(twenty_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo20_cnn_bilstm_roc_auc_Score: %f' % ELMo20_cnn_bilstm_roc_auc_Score)

# Here X_test, y_test are the test data points
predictions =cnn_bilstm_model3.predict(X_test)
predictions = predictions[:, positive]

#Calculating the F1 score of classifier
ELMo30_cnn_bilstm_roc_auc_Score = roc_auc_score(thrity_words_labels, np.round(abs(predictions)), average=weight)
print('ELMo30_cnn_bilstm_roc_auc_Score: %f' % ELMo30_cnn_bilstm_roc_auc_Score)

roc_data = {
    'Name': ['ELMo10_cnn_roc_auc_Score', 'ELMo20_cnn_roc_auc_Score', 'ELMo30_cnn_roc_auc_Score', 'ELMo10_bilstm_roc_auc_Score', 'ELMo20_bilstm_roc_auc_Score', 'ELMo30_bilstm_roc_auc_Score', 'ELMo10_cnn_bilstm_roc_auc_Score', 'ELMo20_cnn_bilstm_roc_auc_Score', 'ELMo30_cnn_bilstm_roc_auc_Score'],
    'Scores': [ELMo10_cnn_roc_auc_Score, ELMo20_cnn_roc_auc_Score, ELMo30_cnn_roc_auc_Score, ELMo10_bilstm_roc_auc_Score, ELMo20_bilstm_roc_auc_Score, ELMo30_bilstm_roc_auc_Score, ELMo10_cnn_bilstm_roc_auc_Score, ELMo20_cnn_bilstm_roc_auc_Score, ELMo30_cnn_bilstm_roc_auc_Score]
}
df = pd.DataFrame(roc_data)
df.to_excel('ELMo_ROC_AUC_Scores_file.xlsx', index=False)

ELMo10_cnn_roc_auc_Score: 0.500000
ELMo20_cnn_roc_auc_Score: 0.500000


ELMo30_cnn_roc_auc_Score: 0.500000


ELMo10_bilstm_roc_auc_Score: 0.500000
ELMo20_bilstm_roc_auc_Score: 0.500000


ELMo30_bilstm_roc_auc_Score: 0.500000


ELMo10_cnn_bilstm_roc_auc_Score: 0.500000
ELMo20_cnn_bilstm_roc_auc_Score: 0.500000
ELMo30_cnn_bilstm_roc_auc_Score: 0.500000
